In [ ]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label
from tabulate import tabulate


#%pip install opencv-python # Required to import cv2 !!!
import cv2

from IPython.display import display, Latex, Math

import os
import time

plt.rcParams['figure.dpi'] = 250

# Task 1

In [ ]:
A_1 = np.array([
    [1, 0],
    [0, 1]
]) 

A_2 = np.array([
    [1, 2],
    [1, 1], 
    [1, 2]
])

A_3 = np.array([
    [2, 1, 1],
    [2, 1, 1], 
    [1, 1, 2]
])

A_4 = np.array([
    [2, 1, 0],
    [1, 2, 3], 
    [0, 3, 3]
])

In [ ]:
def NMF(V, d, delta = 1e-9, maxiter = 1000, seed = 0, store_norms = False):
    """
    ((Skriv en forklaring på hva funksjonen gjør her))
    input:
        V: (m,n) input array
        d: integer, Number of components we want to decompose V into
        delta, float, small number for safe division
        maxiter: integer, maximum number of iterations
        seed: integer, random seed
    output:
        W: (m,d) array
        H: (d,n) array
    """

    if seed != 0:
        np.random.seed(seed)
    
    norms = np.zeros(maxiter)

    #Initialize W and H to random numbers between 0 and 1 with the correct shape
    m = np.shape(V)[0]
    n = np.shape(V)[1]
    W_0 = np.random.uniform(0.0, 1.0, m*d).reshape(m,d)
    H_0 = np.random.uniform(0.0, 1.0, d*n).reshape(d,n)

    #Scale the matrixes
    W_0 = np.sqrt(np.mean(V)/d)*W_0
    H_0 = np.sqrt(np.mean(V)/d)*H_0
    #print("W_0 ",W_0)
    #print("H_0 ",H_0)
    
    W_next = np.copy(W_0)
    W_prev = np.copy(W_0)
    H_next = np.copy(H_0)
    H_prev = np.copy(H_0)
    
    for k in range(maxiter):
        #gir samme resultat
        #H_temp = np.divide(np.multiply(H_prev,np.dot(W_prev.T,V)),np.linalg.multi_dot([W_prev.T,W_prev,H_prev])+delta)
        #W_temp = np.divide(np.multiply(W_prev,np.dot(V,H_next.T)),np.linalg.multi_dot([W_prev,H_next,H_next.T])+delta)

        H_next = (H_prev * np.dot(W_prev.T,V)) / (np.linalg.multi_dot([W_prev.T,W_prev,H_prev])+delta)
        W_next = (W_prev * np.dot(V,H_next.T)) / (np.linalg.multi_dot([W_prev,H_next,H_next.T])+delta)
        H_prev = np.copy(H_next)
        W_prev = np.copy(W_next)
        norms[k] = np.linalg.norm(V- np.dot(W_next, H_next), 'fro')

    if store_norms == False:
        return W_next, H_next
    else:
        return W_next, H_next, norms

## 1d) 

In [ ]:
def test_1d():
    W_next_A_1, H_next_A_1 = NMF(A_1, 1, delta = 1e-9, maxiter = 1000, seed = 0)
    W_next_A_1_new_seed, H_next_A_1_new_seed = NMF(A_1, 1, delta = 1e-9, maxiter = 1000, seed = 8)

    W_next_A_2, H_next_A_2 = NMF(A_2, 1, delta = 1e-9, maxiter = 1000, seed = 1)   
    W_next_A_2_new_seed, H_next_A_2_new_seed = NMF(A_2, 1, delta = 1e-9, maxiter = 1000, seed = 23)    
 

    print("Matrix A_1:")

    data_A11 = [np.array(["W","H","WH","Norm"]),[np.round(W_next_A_1,3),np.round(H_next_A_1,3),np.dot(W_next_A_1,H_next_A_1),np.linalg.norm(A_1- np.dot(W_next_A_1,H_next_A_1), 'fro')]]
    print(tabulate(data_A11, headers='firstrow', tablefmt='fancy_grid'))


    #print(f"W: \n {W_next_A_1}")
    #print(f"H: \n {H_next_A_1}")
    #print(f"WH: \n {np.dot(W_next_A_1,H_next_A_1)}")
    #print(f"Norm: {np.linalg.norm(A_1- np.dot(W_next_A_1,H_next_A_1), 'fro')}") 

    
    print()
    print("Matrix A_1 with new seed:")
    data_A1_new_seed = [np.array(["W","H","WH","Norm"]),[np.round(W_next_A_1_new_seed,3),np.round(H_next_A_1_new_seed,3),np.round(np.dot(W_next_A_1_new_seed,H_next_A_1_new_seed),3),np.linalg.norm(A_1- np.dot(W_next_A_1_new_seed,H_next_A_1_new_seed), 'fro')]]
    print(tabulate(data_A1_new_seed, headers='firstrow', tablefmt='fancy_grid'))
    #print(f"W: \n {W_next_A_1_new_seed}")
    #print(f"H: \n {H_next_A_1_new_seed}")
    #print(f"WH: \n {np.dot(W_next_A_1_new_seed,H_next_A_1_new_seed)}")
    #print(f"Norm: {np.linalg.norm(A_1- np.dot(W_next_A_1_new_seed,H_next_A_1_new_seed), 'fro')}") 
    #print("\n\n")

    

    print("Matrix A_2:")
    # print(f"W: \n {W_next_A_2}")
    # print(f"H: \n {H_next_A_2}")
    # print(f"WH: \n {np.dot(W_next_A_2,H_next_A_2)}")
    # print(f"Norm: {np.linalg.norm(A_2- np.dot(W_next_A_2,H_next_A_2), 'fro')}")

    data_A21 = [np.array(["W","H","WH","Norm"]),[np.round(W_next_A_2,3),np.round(H_next_A_2,3),np.round(np.dot(W_next_A_2,H_next_A_2),3),np.linalg.norm(A_2- np.dot(W_next_A_2,H_next_A_2), 'fro')]]
    print(tabulate(data_A21, headers='firstrow', tablefmt='fancy_grid'))

    #print()
    print("Matrix A_2 new seed:")
    # print(f"W: \n {W_next_A_2_new_seed}")
    # print(f"H: \n {H_next_A_2_new_seed}")
    # print(f"WH: \n {np.dot(W_next_A_2_new_seed,H_next_A_2_new_seed)}")
    # print(f"Norm: {np.linalg.norm(A_2- np.dot(W_next_A_2_new_seed,H_next_A_2_new_seed), 'fro')}")
    data_A2_new_seed = [np.array(["W","H","WH","Norm"]),[np.round(W_next_A_2_new_seed,3),np.round(H_next_A_2_new_seed,3),np.round(np.dot(W_next_A_2_new_seed,H_next_A_2_new_seed),3),np.linalg.norm(A_2- np.dot(W_next_A_2_new_seed,H_next_A_2_new_seed), 'fro')]]
    print(tabulate(data_A2_new_seed, headers='firstrow', tablefmt='fancy_grid'))

    
test_1d()

## 1e

In [ ]:
def test_1e():

    W_next_A_1, H_next_A_1 = NMF(A_1, 2, delta = 1e-9, maxiter = 1000, seed = 0)
    W_next_A_2, H_next_A_2 = NMF(A_2, 2, delta = 1e-9, maxiter = 1000, seed = 0)    
    print("Matrix A_1:")
    data_A1 = [np.array(["W","H","WH","Norm"]),[np.round(W_next_A_1,3),np.round(H_next_A_1,3),np.dot(W_next_A_1,H_next_A_1),np.linalg.norm(A_1- np.dot(W_next_A_1,H_next_A_1), 'fro')]]
    print(tabulate(data_A1, headers='firstrow', tablefmt='fancy_grid'))

    print("Matrix A_2:")
    data_A2 = [np.array(["W","H","WH","Norm"]),[np.round(W_next_A_2,3),np.round(H_next_A_2,3),np.dot(W_next_A_2,H_next_A_2),np.linalg.norm(A_2- np.dot(W_next_A_2,H_next_A_2), 'fro')]]
    print(tabulate(data_A2, headers='firstrow', tablefmt='fancy_grid'))

    #fig = go.Figure(data=[go.Table(header=dict(values=["W","H","WH","Norm"]),
    #             cells=dict(values=[np.round(W_next_A_1,3),np.round(H_next_A_1,3),np.dot(W_next_A_1,H_next_A_1),np.linalg.norm(A_1- np.dot(W_next_A_1,H_next_A_1), 'fro')]))
    #                 ])
    #fig.show()

test_1e()

## 1f)

In [69]:
def oppg_1f():
    eigenvalues_A3,eigenvectors_A3 = np.linalg.eig(A_3)
    eigenvalues_A4,eigenvectors_A4 = np.linalg.eig(A_4)

    #Preparing to round off a the eigenvalue -4.04414586e-16 to 0 instead of -0. Caused by machine error.
    eigenvalues_A3[2] = np.abs(eigenvalues_A3[2])  

    
    display(Latex(f'The three eigenvalues of $A_{3}$ is {np.round(eigenvalues_A3,4)}.')) #Rounds the
    display(Latex(f'The three eigenvalues of $A_{4}$ is {(np.round(eigenvalues_A4,4))}.'))

oppg_1f()


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
def oppg_1g():
    
    W_next_A3_1, H_next_A3_1, norms_A3_1 = NMF(A_3, 1, delta = 1e-9, maxiter = 1000, seed = 0,store_norms=True)
    W_next_A3_2, H_next_A3_2, norms_A3_2 = NMF(A_3, 2, delta = 1e-9, maxiter = 1000, seed = 0,store_norms=True)
    W_next_A3_3, H_next_A3_3, norms_A3_3 = NMF(A_3, 3, delta = 1e-9, maxiter = 1000, seed = 0,store_norms=True)
    iters = np.linspace(0,1000,1000)
    plt.plot(iters,norms_A3_1,label = "d=1")
    plt.plot(iters,norms_A3_2,label= "d=2")
    plt.plot(iters,norms_A3_3,label = "d=3")
    plt.title("A3 for d=1, d=2,d=3")
    plt.yscale('log')
    plt.xlabel("Iterations")
    plt.ylabel(r'$\vert\vert{A-WH}\vert\vert_F$')
    plt.legend()

    plt.show()

    W_next_A4_1, H_next_A4_1, norms_A4_1 = NMF(A_4, 1, delta = 1e-9, maxiter = 1000, seed = 0,store_norms=True)
    W_next_A4_2, H_next_A4_2, norms_A4_2 = NMF(A_4, 2, delta = 1e-9, maxiter = 1000, seed = 0,store_norms=True)
    W_next_A4_3, H_next_A4_3, norms_A4_3 = NMF(A_4, 3, delta = 1e-9, maxiter = 1000, seed = 0,store_norms=True)
    plt.plot(iters,norms_A4_1,label = "d=1")
    plt.plot(iters,norms_A4_2,label= "d=2")
    plt.plot(iters,norms_A4_3,label = "d=3")
    plt.title("A4 for d=1, d=2,d=3")
    plt.yscale('log')
    plt.xlabel("Iterations")
    plt.ylabel(r'$\vert\vert{A-WH}\vert\vert_F$')
    plt.legend()

    plt.show()






oppg_1g()


# Task 2

In [ ]:
# Number of faces to keep in final array
N = 500

In [ ]:
def load_images(N):
    """
    Loads images from cryptopunk dataset. The loading order is deterministic,
    so for a certain N the exact same images will be loaded. 
    Input:
        N, integer, number of images to load
    Output:
        faces, (24,24,4,N) numpy array containing images
    """

    # Allocate array to store images
    faces = np.zeros((24,24,4,N))

    # Iteration variable
    i = 0

    # Iterate over folders
    for subdir, dirs, files in os.walk('./imgs'):

        # Iterate over files
        for file in files:

            # Filepath to load from
            filepath = subdir + os.sep + file

            # Make sure that the file is a .png
            if filepath[-3:] == 'png':

                # Load the image
                im = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)

                # Convert it to RGBA and rescale pixels
                faces[:,:,:,i] = cv2.cvtColor(im, cv2.COLOR_BGRA2RGBA)/255.0

                i+=1
            if i == N:
                break
    return faces

In [ ]:
# Code for plotting images
def plotimgs(imgs, nplot = 8, rescale = False, filename = None):
    """
    Plots nplot*nplot images on an nplot x nplot grid. 
    Saves to given filename if filename is given
    Can also rescale the RGB channels
    input:
        imgs: (24,24,4,N) or (24,24,3,N) array containing images, where N > nplot**2
        nplot: integer, nplot**2 images will be plotted
        rescale: bool
        filename: string, figure will be saved to this location. Should end with ".png".
    """
    # We will change some of the parameters of matplotlib, so we store the initial ones
    oldparams = plt.rcParams['figure.figsize']

    # New params to make better plot. There definitely exists better ways of doing this
    plt.rcParams['figure.figsize'] = (16, 16)

    # Initialize subplots
    fig, axes = plt.subplots(nplot,nplot)

    # Set background color
    plt.gcf().set_facecolor("lightgray")

    # Iterate over images
    for idx in range(nplot**2):
        
        # Indices
        i = idx//nplot; j = idx%nplot

        # Remove axis
        axes[i,j].axis('off')

        # Rescale RGB channels by dividing my maximal value
        if rescale:
            scaled_img = np.copy(imgs[:,:,:,idx])
            scaled_img[:,:,:3] = scaled_img[:,:,:3]/np.max(scaled_img[:,:,:3])
            axes[i,j].imshow(scaled_img)
        else:
            axes[i,j].imshow(imgs[:,:,:,idx])
    
    # Tight layout so images will appear closer together
    plt.tight_layout()

    # Save if filename is given
    if filename is not None:
        plt.savefig(filename)

    plt.show()

    # Return to old parameters
    plt.rcParams['figure.figsize'] = oldparams

## 2a

In [ ]:
def plot_random_faces():
    # Load all 10000 images
    faces = load_images(10000)

    # Select N random images
    choices = np.random.choice(faces.shape[-1],N, replace = False)
    faces = faces[:,:,:,choices]

    # Check that the shape is correct
    print(faces.shape)

    # Example of plotting 8 times 8 images stored in "faces" and saving the output to a file named "punks.png"
    plotimgs(faces, 8, filename="punks.png")
plot_random_faces()

## 2b
Jeg har funnet ut at det er raskere å hente bildedata fra en npz fil vs loadimages, så jeg gjør det


In [ ]:
def generate_face_data():
    '''
    This funtion loads all images to numpy arrays and saves them to a selection of smaller arrays. This
    is to reduce runtime while testing the upcomming calculations.
    '''
    # Load all 10000 images
    faces = load_images(10000)

    # Select 500 random images
    N = 500
    choices = np.random.choice(faces.shape[-1],N, replace = False)
    faces_500 = faces[:,:,:,choices]
    assert (faces_500.shape == (24, 24, 4, 500))  # Make sure the dimensions are correct
    
    # Select 100 random images
    N = 100
    choices = np.random.choice(faces.shape[-1],N, replace = False)
    faces_100 = faces[:,:,:,choices]
    assert (faces_100.shape == (24, 24, 4, 100))  # Make sure the dimensions are correct
    
    # Select 100 random images
    N = 4
    choices = np.random.choice(faces.shape[-1],N, replace = False)
    faces_4 = faces[:,:,:,choices]
    assert (faces_4.shape == (24, 24, 4, 4))  # Make sure the dimensions are correct

    
    np.savez(
        'faces',
        faces = faces,
        faces_500 = faces_500,  # 500 faces
        faces_100 = faces_100,  # 100 faces
        faces_4 = faces_4       # 4 faces
    )
generate_face_data()

In [ ]:
def find_mean_faces(face_data):
    '''
    Contrary to the name, this function does not actualy find 'mean' faces, but rather the mean value of each pixel
    in the faces dataset.
    Input:
        face_data: string, the face array for which we will calculate the mean
    The output is saved to a file as a 24*24*4 array:
        Filename:   mean_faces.npz
        Arraydata:  face_mean
    '''
    # Start by loading the faces that are to be used
    faces_data = np.load('faces.npz')
    faces = faces_data[face_data]

    ########################################################
    # Create some test-data to verify that the 
    # method works
    # faces[:, :, :, 0] = [[[1/4, 0, 0, 1 ]]]   # red red
    # faces[:, :, :, 1] = [[[1/4, 0, 0, 1 ]]]   # red red
    # faces[:, :, :, 2] = [[[1/4, 0, 0, 1 ]]]   # red red
    # faces[:, :, :, 3] = [[[1/4, 0, 0, 1 ]]]   # nothing

    # Alter face 3
    # faces[6:18, 6:18, :, 3] = [[[0, 1, 0, 1]]]  # green
    # The mean should be yellow

    
    plotimgs(faces, 8)
    ########################################################


    # The images are stored on the last index of the faces array
    mean_faces = np.mean(faces, axis = 3)
    assert(mean_faces.shape == (24, 24, 4)) # assert the correct shape
    mean_faces = np.reshape(mean_faces, (24, 24, 4, 1))
    
    # Plotting
    plt.imshow(mean_faces[:, :, :, 0])
    plt.axis('off')
    plt.show()

find_mean_faces('faces')

## 2c

In [ ]:
def calculate_NMF(d_val, faces_name):  
    '''
    Calculating the NMF of the 3 color channels, and plotting the the columns of W interpreted as RGB images.

    Input:
        d_val: int, number of columns in the W-matrix
        
    Output:
        W: array
        H: array
    '''
    
    # Loading the image-data
    faces_data = np.load('faces.npz')

    ################################################# OLD CODE
    #faces = faces_data['faces_500'][:, :, 0:3, :]
    # faces = faces_data[faces_name][:, :, 0:3, :]
    # assert (faces.shape == (24, 24, 3, 500))
    
    # # Reshape the remaining three color channels into a 1728xN
    # faces_reshaped = np.reshape(faces, (1728, 500))
    # assert (faces_reshaped.shape == (1728, 500))
    ################################################# 

    faces = faces_data[faces_name]

    # Split the images into opacity and rgb channels
    faces_opacity = faces[:,:,3,:]
    faces_rgb = faces[:,:,:3,:]
    shape = faces_rgb.shape

    # Clever reshape "trick". This is the matrix we will apply the NMF to
    faces_reshaped = faces_rgb.reshape(np.prod(shape)//N, N)

    # Calculate the NMF of the A-matrix
    W, H = NMF(faces_reshaped, d_val)

    # Reshaping the W-matrix into a 24×24×3x64-matrix
    W = np.reshape(W, (24, 24, 3, d_val))
    assert (W.shape == (24, 24, 3, d_val))

    # Plot the columns of W 
    plotimgs(W, nplot = 8, rescale = False, filename = None)

    # Returning the W- and H-matrixes
    return W, H

W_calculated, H_calculated = calculate_NMF(64, 'faces_500')

## 2d

In [ ]:
def calculate_reconstructions(W, H, plot_original_faces = False, original_faces_name = ''):
    '''
    Calculating the reconstructions WH, and plotting these

    Input:
        d_val: int, number of columns in the W-matrix
        
    Output:
        W: array, previously calculated W-matrix
        H: array, previously calculated W-matrix
    '''
    if plot_original_faces == True:
        faces_data = np.load('faces.npz')
        # faces = faces_data['faces_500']
        faces = faces_data[original_faces_name]
        plotimgs(faces, nplot = 8, rescale = False, filename = None)

    # Plotting the matrix A = WH using the plotimgs-funtion.
    plotimgs(np.dot(W, H), nplot = 8, rescale = False, filename = None)
    
# calculate_reconstructions(W_calculated, H_calculated)

## 2e

In [ ]:
def oppgave_to_e():
    d_vals = [16, 32, 64]
    norms = []
    
    # Loading the image-data
    faces_data = np.load('faces.npz')
    faces = faces_data['faces_500'][:, :, 0:3, :]
    assert (faces.shape == (24, 24, 3, 500))
    
    # Reshape the remaining three color channels into a 1728xN
    faces_reshaped = np.reshape(faces, (1728, 500))
    assert (faces_reshaped.shape == (1728, 500))

    for i in range(len(d_vals)):

        W, H, n = NMF(faces_reshaped, d_vals[i], delta = 1e-9, maxiter = 1000, seed = 0, store_norms = True)
        #np.append(norms, norm)
        norms.append(n)

    return norms
        
#norms_calculated = oppgave_to_e()

In [ ]:
def oppgave_to_e_2(norms):
    x_arr = np.arange(0, len(norms[0]), 1)
    plt.plot(x_arr,norms[0], label = 'd = 16')
    plt.plot(x_arr,norms[1], label = 'd = 32')
    plt.plot(x_arr,norms[2], label = 'd = 64')
    plt.title("Norms as function of iterations")
    plt.xlabel(r'$k$')
    plt.ylabel(r'$\vert\vert{A-W_{k} H_{k}}\vert\vert_{F}$')
    plt.legend()
    plt.yscale('log')
    plt.show()
#oppgave_to_e_2(norms_calculated)

# Task 3

Det som er skrevet under er kopiert fra utdelt kode

In [ ]:
# Legge til støy
def add_noise(imgs_reshaped, sigma = 0.1):
    """
    Adds gaussian noise to images as described in text.
    Note that imgs must be given as a (24*24*3, N) numpy array, i.e the reshaped images
    Input:
        imgs_reshaped: (1728,N) numpy array
        sigma: scalar, noise level
    Output:
        noisy_faces: (1728,N) numpy array containing noisy images
    """
    # Noise level
    sigma = 0.1

    # Array that will store the rgb channels of the noisy images
    noisy_faces = np.copy(imgs_reshaped)

    # Number of noisy values we need
    nnzero = faces_reshaped[np.nonzero(imgs_reshaped)].shape[0]

    # Sample noisy values and add noise
    noise = np.random.normal(0.0,1,nnzero)
    noisy_faces[np.nonzero(imgs_reshaped)] += sigma*noise

    # Clip to lie between 0 and 1 so that we can still interpret them as images
    noisy_faces = np.maximum(0.0,np.minimum(1.0, noisy_faces))

    return noisy_faces

In [ ]:
noisy_faces = add_noise(faces_reshaped)

# Calculate the error of the noisy images
noise_residual = np.linalg.norm(noisy_faces - faces_reshaped)
print(noise_residual)

# For plotting noisy images we add the opacity
noisy_faces_with_opacity = np.zeros(faces.shape)
noisy_faces_with_opacity[:,:,:3,:] = noisy_faces.reshape(faces_rgb.shape)
noisy_faces_with_opacity[:,:,3,:] = faces_opacity
plotimgs(noisy_faces_with_opacity)